In [8]:
import os, re, glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import shutil

groups_folder_path = 'C:/Users/jhlee/Desktop/code/melspec/'# need to add path
categories = ["001", "002", "003"] # categories
 
num_classes = len(categories)
  
image_w = 28
image_h = 28
  
X = [] #image save array
Y = [] #label array
name= [] # image array

for idex, categorie in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    image_dir = groups_folder_path + categorie + '/'
    for top, dir, f in os.walk(image_dir):
        for filename in f:
            print(image_dir+filename)
            img = cv2.imread(image_dir+filename)
            img = cv2.resize(img, None, fx=image_w/img.shape[1], fy=image_h/img.shape[0])
            X.append(img/256)
            Y.append(label)

print(len(X))

X = np.array(X)
Y = np.array(Y)

print(len(X))

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
xy = (X_train, X_test, Y_train, Y_test)
 
np.save("./img_data.npy", xy)


C:/Users/jhlee/Desktop/code/melspec/001/0.jpg
C:/Users/jhlee/Desktop/code/melspec/001/0_0.jpg
C:/Users/jhlee/Desktop/code/melspec/001/1.jpg
C:/Users/jhlee/Desktop/code/melspec/001/10.jpg
C:/Users/jhlee/Desktop/code/melspec/001/100.jpg
C:/Users/jhlee/Desktop/code/melspec/001/101.jpg
C:/Users/jhlee/Desktop/code/melspec/001/102.jpg
C:/Users/jhlee/Desktop/code/melspec/001/103.jpg
C:/Users/jhlee/Desktop/code/melspec/001/104.jpg
C:/Users/jhlee/Desktop/code/melspec/001/105.jpg
C:/Users/jhlee/Desktop/code/melspec/001/106.jpg
C:/Users/jhlee/Desktop/code/melspec/001/107.jpg
C:/Users/jhlee/Desktop/code/melspec/001/108.jpg
C:/Users/jhlee/Desktop/code/melspec/001/109.jpg
C:/Users/jhlee/Desktop/code/melspec/001/11.jpg
C:/Users/jhlee/Desktop/code/melspec/001/110.jpg
C:/Users/jhlee/Desktop/code/melspec/001/111.jpg
C:/Users/jhlee/Desktop/code/melspec/001/112.jpg
C:/Users/jhlee/Desktop/code/melspec/001/113.jpg
C:/Users/jhlee/Desktop/code/melspec/001/114.jpg
C:/Users/jhlee/Desktop/code/melspec/001/115.jp

In [16]:
import os, re, glob
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from tensorflow.keras.layers import Flatten, Convolution2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import *
import cv2
 
X_train, X_test, Y_train, Y_test = np.load('./img_data.npy', allow_pickle=True)
 
model = Sequential()
model = Sequential()
model = Sequential()
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))  
model.add(Dense(num_classes))
model.add(Activation('softmax'))  
  
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

history = model.fit(X_train, Y_train, batch_size=32, epochs=100)

model.summary()


model.save('myFirstCNN.h5')



Epoch 1/100
13/13 [==============================] - 1s 5ms/step - loss: 0.5947 - accuracy: 0.6348
Epoch 2/100
13/13 [==============================] - 0s 6ms/step - loss: 0.1972 - accuracy: 0.9572
Epoch 3/100
13/13 [==============================] - 0s 6ms/step - loss: 0.1014 - accuracy: 0.9950
Epoch 4/100
13/13 [==============================] - 0s 5ms/step - loss: 0.0655 - accuracy: 0.9975
Epoch 5/100
13/13 [==============================] - 0s 5ms/step - loss: 0.0368 - accuracy: 1.0000
Epoch 6/100
13/13 [==============================] - 0s 4ms/step - loss: 0.0247 - accuracy: 1.0000
Epoch 7/100
13/13 [==============================] - 0s 4ms/step - loss: 0.0237 - accuracy: 1.0000
Epoch 8/100
13/13 [==============================] - 0s 5ms/step - loss: 0.0146 - accuracy: 1.0000
Epoch 9/100
13/13 [==============================] - 0s 5ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 10/100
13/13 [==============================] - 0s 5ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 11/

X_train = np.append(X_train,X_test, axis=0)
Y_train = np.append(Y_train,Y_test, axis=0)
 
# Save Model with CheckPoint & StopPoint
from keras.callbacks import ModelCheckpoint,EarlyStopping
import os
import datetime
 
Datetime = datetime.datetime.now().strftime('%m%d_%H%M')
modelpath="Gersang.h5"
 
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='loss', patience=100)
 
# Learning and save models
model.fit(X_train, Y_train, validation_split=0.1, epochs=3500, batch_size=10, verbose=0, callbacks=[early_stopping_callback,checkpointer])



In [56]:
import os, re, glob
import cv2
import numpy as np
import shutil
from numpy import argmax
from keras.models import load_model
 
categories = ["001", "002", "003"] # categories
 
def Dataization(img_path):
    image_w = 28
    image_h = 28
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=image_w/img.shape[1], fy=image_h/img.shape[0])
    return (img/256)
 
src = []
name = []
test = []
image_dir = "C:/Users/jhlee/Desktop/code/test_sample/"
for file in os.listdir(image_dir):
    if (file.find('.jpg') != -1):
           
        src.append(image_dir + file)
        name.append(file)
        test.append(Dataization(image_dir + file))
 

test = np.array(test)
model = load_model('myFirstCNN.h5')
predict_x = model.predict(test)
classes_x = np.argmax(predict_x,axis=1)

#predict = model.predict_classes(test)

one = 0
two = 0
three = 0



for i in range(len(test)):
    print(name[i] + " : , Predict : "+ str(categories[classes_x[i]]))
    if(str(categories[classes_x[i]]) == '001'):
        one += 1
    if(str(categories[classes_x[i]]) == '002'):
        two += 1
    if(str(categories[classes_x[i]]) == '003'):
        three += 1

print(one)
print(two)
print(three)

0.jpg : , Predict : 002
1.jpg : , Predict : 002
10.jpg : , Predict : 002
11.jpg : , Predict : 002
12.jpg : , Predict : 002
13.jpg : , Predict : 002
14.jpg : , Predict : 002
15.jpg : , Predict : 002
16.jpg : , Predict : 002
17.jpg : , Predict : 001
18.jpg : , Predict : 001
19.jpg : , Predict : 002
2.jpg : , Predict : 002
20.jpg : , Predict : 002
21.jpg : , Predict : 002
22.jpg : , Predict : 002
23.jpg : , Predict : 002
24.jpg : , Predict : 002
25.jpg : , Predict : 001
26.jpg : , Predict : 001
27.jpg : , Predict : 001
28.jpg : , Predict : 001
29.jpg : , Predict : 001
3.jpg : , Predict : 002
30.jpg : , Predict : 001
31.jpg : , Predict : 001
32.jpg : , Predict : 001
33.jpg : , Predict : 002
34.jpg : , Predict : 002
35.jpg : , Predict : 002
36.jpg : , Predict : 001
37.jpg : , Predict : 001
38.jpg : , Predict : 002
4.jpg : , Predict : 002
5.jpg : , Predict : 002
6.jpg : , Predict : 002
7.jpg : , Predict : 002
8.jpg : , Predict : 002
9.jpg : , Predict : 002
12
27
0
